## Задача 1. Оценка эксперимента с CUPED

Оцените эксперимент «Sending email (correct link)» с использованием CUPED. В качестве ковариаты используйте выручку пользователей за 4 недели до эксперимента.

Данные эксперимента «Sending email (correct link)»: "2022-05-03/2022-05-03T12_df_sales.csv" и "2022-05-03/experiment_users.csv". Эксперимент проводился с 2022-04-25 по 2022-05-02. Метрика — средняя выручка с клиента.

В качестве ответа введите p-value с точность до 4-го знака после точки.

In [1]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from datetime import datetime, timedelta


URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-05-03/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

sales_df = read_database('2022-05-03T12_df_sales.csv')
users_df = read_database('experiment_users.csv')


In [2]:
len(users_df)

109367

In [3]:
exp_start_dt = datetime(2022, 4, 25)
exp_end_dt = exp_start_dt + timedelta(days=7)
pre_period_start = exp_start_dt - timedelta(weeks=4)

In [4]:
sales_df['date'] = sales_df.apply(lambda x: pd.to_datetime(x['date']), axis=1)

sales_exp = sales_df[
    (sales_df['date'] >= exp_start_dt)
    &
    (sales_df['date'] < exp_end_dt)
].copy()

sales_pre = sales_df[
    (sales_df['date'] >= pre_period_start)
    &
    (sales_df['date'] < exp_start_dt)
].copy()

In [5]:
users_exp = users_df.merge(sales_exp, how='left', on='user_id')
users_exp['price'] = users_exp['price'].fillna(0).astype('int')
users_exp = users_exp.groupby(['user_id', 'pilot']).price.sum().reset_index()

users_pre = users_df.merge(sales_pre, how='left', on='user_id')
users_pre['price'] = users_pre['price'].fillna(0).astype('int')
users_pre = users_pre.groupby(['user_id', 'pilot']).price.sum().reset_index()
users_pre = users_pre.rename({
    'price': 'cov'
}, axis=1)

In [6]:
exp_x_pre_df = users_exp.merge(users_pre, how='inner', on=['user_id', 'pilot'])

In [7]:
users_test = exp_x_pre_df[exp_x_pre_df['pilot'] == 1].copy()
users_control = exp_x_pre_df[exp_x_pre_df['pilot'] == 0].copy()


In [8]:
print(users_test.price.mean(), users_control.price.mean())

321.47956288554116 313.2905217772581


In [9]:
print(len(users_test) + len(users_control))

109367


In [10]:
help_df = pd.read_csv('df_metrics_1000.csv')

In [11]:
help_df = help_df.rename({
    'pilot': 'pilot_help',
    'metric': 'metric_help',
    'cov': 'cov_help'
}, axis=1)

In [12]:
tmp_res = pd.concat([users_test, users_control])

In [13]:
tmp_res = tmp_res.merge(help_df, how='inner', on='user_id')

In [14]:
tmp_res[
    (tmp_res['pilot'] != tmp_res['pilot_help'])
    |
    (tmp_res['price'] != tmp_res['metric_help'])
    |
    (tmp_res['cov'] != tmp_res['cov_help'])
]

,user_id,pilot,price,cov,pilot_help,metric_help,cov_help


In [52]:
def calculate_teta(df):
    covariance = np.cov(df['cov'], df['price'])[0, 1]
    variance = np.var(df['cov'])
    print(covariance, variance)
    return covariance / variance

In [53]:
common_theta = calculate_teta(pd.concat([users_test, users_control]))

164508.86473543497 1860131.18327586


In [47]:
def calculate_cuped_metric(df):
    
    teta = common_theta
    
    x_predict = calc_predict(df)

    df['cuped_metric'] = df.apply(lambda x: x['price'] - teta * x['cov'], axis=1)
    
    return df[['cuped_metric']]

In [48]:
test_cuped = calculate_cuped_metric(users_test.copy())
control_cuped = calculate_cuped_metric(users_control.copy())


In [49]:
print(len(test_cuped), len(control_cuped))

54631 54736


In [50]:
_, pvalue = stats.ttest_ind(test_cuped, control_cuped)

In [51]:
pvalue

array([0.05394612])

## Задача 2. Функция вычисления CUPED-метрики

Реализуйте функцию calculate_cuped_metric.

Обратите внимание, что в этом задании нужно использовать формулу из лекции с вычитанием среднего значения ковариаты.

Внимание!
Для вычисления параметра θ нужно оценить ковариацию и дисперсию. Оценивать их можно разными способами, например, есть смещённые и несмещённые оценки. На выборках маленького размера значения разных оценок могут сильно отличаться. Для решения этого задания используйте функции из библиотеки numpy с параметрами по умолчанию.

In [23]:
import numpy as np
import pandas as pd


def calculate_teta(df):
    return np.cov(df['metric'], df['cov'])[0, 1] / np.var(df['cov'])


def calculate_cuped_metric(df_metric, df_cov):
    """Считает значения cuped-метрики.

    :param df_metric (pd.DataFrame): таблица со значениями метрики во время эксперимента
        со столбцами ['user_id', 'metric'].
    :param df_cov (pd.DataFrame): таблица со значениями ковариаты
        со столбцами ['user_id', 'cov'].
    :return df: таблица со значениями cuped-метрики со столбцами ['user_id', 'metric'].
    """
    
    metric_x_cov = df_metric.merge(df_cov, how='left', on='user_id')
    
    print(metric_x_cov)
    
    teta = calculate_teta(metric_x_cov)
    mean_x = df_cov['cov'].sum() / len(df_cov)
    
    print(teta, mean_x)
    
    metric_x_cov['metric'] = metric_x_cov.apply(lambda x: x['metric'] - teta * (x['cov'] - mean_x), axis=1)
    
    return metric_x_cov[['user_id', 'metric']]
    
    
    

In [24]:
df_metric = pd.DataFrame({'user_id': [1, 2, 3], 'metric': [2000, 2500, 3000]})
df_cov = pd.DataFrame({'user_id': [1, 2, 3], 'cov': [1100, 1500, 0]})

df = calculate_cuped_metric(df_metric, df_cov)
# df = pd.DataFrame({'user_id': [1, 2, 3], 'metric': [2159.53, 2933.01, 2407.46]})

   user_id  metric   cov
0        1    2000  1100
1        2    2500  1500
2        3    3000     0
-0.6837016574585636 866.6666666666666


In [25]:
print(df)

   user_id       metric
0        1  2159.530387
1        2  2933.011050
2        3  2407.458564
